# Tensorflow Introduction

Tensorflow is a library which allows us to specify functions as computational graphs. Since Neural Nets are really just functions specified as graphs, we can use Tensorflow to deal with the low-level details of Neural Nets while we focus on the high-level architecture of the network we want to use. 

To start, we'll go through the basics of Tensorflow: how to use it to define computational graphs, and how to train these graphs with backpropogation to mimimize an error. After we have a hold on the basics, in the next workshop we'll use Tensorflow to create and train a neural net on the classic handwritten digits classification task. 



In [1]:
#let's start by importing tensorflow
import tensorflow as tf

## 1. Basics

There are two key kinds of objects (amoung others) which Tensorflow deals with. The first is a "placeholder". Think of this as a kind of input variable which we have yet to specify its value. For example, $x$ is a placeholder in the expression $3x^2+5x+1$. We haven't yet specified what $x$ is yet, but once we do, we know how to evaluate the expression. 

The second kind of object is a "variable". Variables have definite, initialized values. We might try to change these values in order to reduce the error of our function on some training set. For example, imagine our function is $f(x)=ax+b$, where $x$ is a placeholder and $a,b$ are variables. We initialize $a,b$ to be values taken from a normal distribution. Now imagine we have some data $X=[1,2,3,4]$ and $Y=[2,3,4,5]$. If we represent $f$ as a Tensorflow graph, and "train" $f$ on the data $X$ and $Y$, this means that we want to try and find values of $(a,b)$ such that $f(1)$ is close to $2$, $f(2)$ is close to 3 etc. (i.e., essentially linear regression). In other words, we optimize the fit of our function over "variables", and "placeholders" specify our function input. Here's an example of how to use Tensorflow to define the function $f(x)=ax^2+bx+1$:

In [2]:
#our input x is a float, of tensor dimension [None,1]. The None specifies that we might pass any number of input
#points to x, while the 1 specifies that x is a scalar (i.e. 1 dimensional)
x = tf.placeholder(tf.float32, [None,1])
#each of our variables are initialized to a random number 
a=tf.Variable(tf.random_normal([1],stddev=40.0))
b=tf.Variable(tf.random_normal([1],stddev=40.0))
c=tf.Variable(tf.random_normal([1],stddev=40.0))
#y is like our y=f(x): we tell Tensorflow how to combine a,b,c,x to get f(x)
y=tf.add(tf.add(tf.multiply(a,tf.multiply(x,x)),tf.multiply(b,x)),c)
#Note: instead of the above lengthly expression, we can use the shorthand y=a*x*x+b*x+c


#let's run this function on input [1],[2],[3],[4]: that is, we want to compute f([1]), f([2]),.... 
#first, we need to initialize all our variables and run a session. Why all the extra code? 
#This is because Tensorflow doesn't actually run the model in Python (python is just a front-end); this is so that
#your code can run faster

X=[[1],[2],[3],[4]]
Target=[[-1],[2],[3],[4]]
#code we have to call every time
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
#let's get our ouput
output=sess.run(y,feed_dict={x: X})
print(output)






[[ -39.16268921]
 [ -84.15621948]
 [-179.24130249]
 [-324.41793823]]


In [3]:
#Write a function to plot output and Target on the same axis




In [4]:
#This probably isn't a very good fit, so let's train our model
#First, we need to define an error function. Let's start with the mean squared error

#We define an error function in exactly the same way we defined f(x) above. Our error function takes 
#y=the output of f(x), and a new variable y_=the target value we want y to be. 
#First, we initialize a placeholder y_ to hold our target answers (just like we initialized x before)
y_=tf.placeholder(tf.float32,[None,1])
#define the error function
error=tf.reduce_mean(tf.multiply(tf.subtract(y_, y),tf.subtract(y_,y)))
#the above is equivalent to the average of (y_-y)^2

#Here's the current error on the dataset
print(sess.run(error,feed_dict={x: X,y_:Target}))




37487.4


## 2. Training the model

Now that we've defined an error function, we want to try and minimize the error of our model with respect to our data. Tensorflow has some built-in methods to handle gradient descent for us. However, this example (depending on the random values of $a,b,c$ which were initialized) might give you some really useful practical advice on the dangers of blindly applying an optimizer to a problem. 

The lines of code below create a gradient optimizer for the function we have defined. The numerical argument for the optimizer specifies its learning rate: the greater this value, the more the optimizer will try to move the parameters $(a,b,c)$ at each step of the optimization procedure. A higher learning rate can mean faster optimization, but there's also a danger that too high a learning rate will cause the optimizer to "overreact" to the gradient at each step, causing some pretty bad behaviour. 

The for loop runs the optimizer for a number of steps. You should write a function to print out the error of the model after the for loop has finished, and then plot the updated model on the data along with the target values (as you did before). You should re-run this code block multiple times and experiment with different values of the learning rate. Try to find a trade-off between learning rate and the number of steps it takes to train the model reasonably well. If you need to re-initialize the parameters $(a,b,c)$ in your model (because the optimizer did something to them you didn't like, like setting them to infinity because the learning rate was too high), you can re-run the code block where the model was initialized to reset the values.


In [5]:
#now let's try to reduce that error

train_step = tf.train.GradientDescentOptimizer(5).minimize(error)
for i in range(1):
    sess.run(train_step, feed_dict={x: X,y_:Target})
#print the current values of (a,b,c)
print("a is", sess.run(a))
print("b is", sess.run(b))
print("c is", sess.run(c))
    
#Now write functions to get the current error of the model, and plot the graph of the model and target on the data




('a is', array([ 18169.07226562], dtype=float32))
('b is', array([ 5207.32080078], dtype=float32))
('c is', array([ 1543.18457031], dtype=float32))


## 3. Getting fancy with Tensors

So far, we've only been working with scalars. In fact, you may have been wondering why we pass our input data as e.g. $x=[5]$ rather than $x=5$. This is because $x$ is really a 1 dimensional tensor. This framework allows us to easily entend the complexity of our models without many changes. For example, instead of multiplying scalars, we can multiply matricies. Here's an example of how this might be done:

In [6]:


#let's create a model which takes a 10 dimensional vector v as input, and returns soft_max(vM+b)
input_v=tf.placeholder(tf.float32,[None,10])
MatrixVariable=tf.Variable(tf.random_normal([10,10],stddev=40.0))
b_variable=tf.Variable(tf.random_normal([10],stddev=40.0))
output_vector=tf.nn.softmax(tf.matmul(input_v,MatrixVariable) + b_variable)

#sample input
sample_input=[[1,2,3,4,5,6,7,8,9,10]]

#run the model
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
output=sess.run(output_vector,feed_dict={input_v: sample_input})
print(output)
sess.close()


[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]]


## 4. Make your own model and dataset

Now that you know how tensorflow works, it's time to make your own model. Generate your own dataset, come up with a model, code it in tensorflow, and then train you model on the dataset you made. Be sure to explain:

1. your data generation process
2. your model
3. your error function

When you've defined the above, train your model on your dataset, and find a way to visualize the performance of your fitted model (for example, by plotting the error of your model over time, or evaluating your fitted model on a test set). If you want to use functions which are more complicated than matrix multiplication, or you want to know how to implement a particular function (e.g. averaging) in Tensorflow, you should look at the Tensorflow docs online. 



In [7]:
##########code your data generation process






#########code your model


#########code your error function



########code your training procedure



#######test your model


